In [1]:
import pandas as pd
import numpy as np

# Transcript

In [2]:
transcript = pd.read_csv("../Data/transcript.csv",index_col=0)
transcript = transcript.rename(columns={'person': 'person id'})
transcript

,person id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [3]:
transcript.isnull().sum()

person id    0
event        0
value        0
time         0
dtype: int64

## drop all the transactional rows

In [4]:
transcript.drop(transcript.index[transcript['event'] == "transaction"], inplace=True)
transcript.event.unique()

array(['offer received', 'offer viewed', 'offer completed'], dtype=object)

In [5]:
transcript.shape

(167581, 4)

## get the offer ids from 'value' column

In [6]:
import json
transcript['offer id'] = [list(json.loads(i.replace("'",'"')).values())[0] for i in transcript['value']]
transcript.head()

,person id,event,value,time,offer id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [7]:
transcript = transcript.drop(columns = 'value')
transcript = transcript.drop(columns = 'time')
transcript.head()

,person id,event,offer id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0


## - there should be no null values.  offer id for those weren't extracted as offer id was writted as 'offer_id' instead of 'offer id'. they should not be dropped

In [ ]:
#transcript.dropna(inplace=True)
transcript

In [ ]:
transcript['event'].value_counts()

# Profile

In [8]:
profile = pd.read_csv("../Data/profile.csv",index_col=0)
profile.rename(columns={"id":"person id"}, inplace=True)
profile.head()

,gender,age,person id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [9]:
profile.isnull().sum()

gender              2175
age                    0
person id              0
became_member_on       0
income              2175
dtype: int64

In [10]:
profile.dropna(inplace=True)
profile.isnull().sum()

gender              0
age                 0
person id           0
became_member_on    0
income              0
dtype: int64

## create member since column

In [11]:
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile['member_since'] = (pd.to_datetime('2021-01-01') - profile['became_member_on']).astype('timedelta64[D]').astype(int)
profile.head()

,gender,age,person id,became_member_on,income,member_since
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1266
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1333
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,981
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,1057
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,1147


# Porfolio

In [12]:
portfolio = pd.read_csv("../Data/portfolio.csv", index_col=0)
portfolio.rename(columns = {'id':'offer id'}, inplace=True)
portfolio.drop(columns = ['reward', 'difficulty', 'duration','channels'], inplace = True)
portfolio

,offer_type,offer id
0,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,informational,3f207df678b143eea3cee63160fa8bed
3,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,discount,fafdcd668e3743c1bb461111dcafc2a4
7,informational,5a8bc65990b245e5a138643cd4eb9837
8,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,discount,2906b810c7d4411798c6938adc9daaa5


# Merge Datasets

In [13]:
df = pd.merge(transcript,profile,on='person id')
df

,person id,event,offer id,gender,age,became_member_on,income,member_since
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333
3,78afa995795e4d85b5d9ceeca43f5fef,offer received,5a8bc65990b245e5a138643cd4eb9837,F,75,2017-05-09,100000.0,1333
4,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,5a8bc65990b245e5a138643cd4eb9837,F,75,2017-05-09,100000.0,1333
...,...,...,...,...,...,...,...,...
148800,c207a8a8ef5c47ee970ca45f859e07eb,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,F,52,2017-12-23,86000.0,1105
148801,c207a8a8ef5c47ee970ca45f859e07eb,offer completed,2298d6c36e964ae4a3e7e9706d1fb8c2,F,52,2017-12-23,86000.0,1105
148802,9a3f45cf29ef428b932492c7a5d6ac52,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,M,20,2016-10-19,70000.0,1535
148803,9a3f45cf29ef428b932492c7a5d6ac52,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,M,20,2016-10-19,70000.0,1535


In [14]:
df = pd.merge(df,portfolio, on='offer id')
df

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo
...,...,...,...,...,...,...,...,...,...
148800,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,2017-10-13,94000.0,1176,bogo
148801,3f3494f3234e4206a279662bef81fe85,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo
148802,3f3494f3234e4206a279662bef81fe85,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo
148803,3045af4e98794a04a5542d3eac939b1f,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,2016-10-20,78000.0,1534,bogo


In [15]:
df.shape

(148805, 9)

# Create dummy variables

## did user redeemed atleast one offer

In [16]:
#get the list of users that completed offers
redm_users=np.array(df[df['event']=='offer completed']['person id'])
len(np.unique(redm_users))

11986

In [17]:
#get unique users that completed offers
uredm_users=np.unique(redm_users)
uredm_users

array(['0009655768c64bdeb2e877511632db8f',
       '0011e0d4e6b944f998e987f904e8c1e5',
       '0020c2b971eb4e9188eac86d93036a77', ...,
       'fff7576017104bcc8677a8d63322b5e1',
       'fffad4f4828548d1b5583907f2e9906b',
       'ffff82501cea40309d5fdd7edcca4a07'], dtype=object)

In [18]:
#if user id is in the list of unique users that redeemed an offer, set redeemed to True
df['redeemed'] = df.apply(lambda x: int(x['person id'] in uredm_users), axis=1)
df.head()

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1


## gender

### - need a dummy variable for male as well

In [19]:
#dummies for gender
df['female'] = [1 if i=='F' else 0 for i in df['gender']]
df['male'] = [1 if i=='M' else 0 for i in df['gender']]
df

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed,female,male
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
148800,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,2017-10-13,94000.0,1176,bogo,0,0,1
148801,3f3494f3234e4206a279662bef81fe85,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0
148802,3f3494f3234e4206a279662bef81fe85,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0
148803,3045af4e98794a04a5542d3eac939b1f,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,2016-10-20,78000.0,1534,bogo,0,1,0


## offer type

### - we're interested in bogo and discount, not informational

### received

In [20]:
#dummies for offer type
df['rc_bogo'] = [1 if i=='bogo' else 0 for i in df['offer_type']]
#df['informational'] =[1 if i=='informational' else 0 for i in df['offer_type']]
df['rc_discount'] =[1 if i=='discount' else 0 for i in df['offer_type']]
df

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed,female,male,rc_bogo,rc_discount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,2017-10-13,94000.0,1176,bogo,0,0,1,1,0
148801,3f3494f3234e4206a279662bef81fe85,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0,1,0
148802,3f3494f3234e4206a279662bef81fe85,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0,1,0
148803,3045af4e98794a04a5542d3eac939b1f,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,2016-10-20,78000.0,1534,bogo,0,1,0,1,0


### redeemed

In [21]:
df['rd_bogo'] = df.apply(lambda x: int((x.event=='offer completed') and (x.offer_type == 'bogo')), axis=1)
df['rd_discount'] = df.apply(lambda x: int((x.event=='offer completed') and (x.offer_type == 'discount')), axis=1)
df

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,0,0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,0,0
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,1,0
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0,0,0
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,2017-10-13,94000.0,1176,bogo,0,0,1,1,0,0,0
148801,3f3494f3234e4206a279662bef81fe85,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0,1,0,0,0
148802,3f3494f3234e4206a279662bef81fe85,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0,1,0,0,0
148803,3045af4e98794a04a5542d3eac939b1f,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,2016-10-20,78000.0,1534,bogo,0,1,0,1,0,0,0


In [22]:
df.redeemed.value_counts()

1    128773
0     20032
Name: redeemed, dtype: int64

In [23]:
df.sample(30)

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
136827,92fbbcb6323f43b087bb5e9ae6ee1eca,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,F,46,2017-09-23,33000.0,1196,bogo,1,1,0,1,0,0,0
67055,e83e3e6bdad14414afee38668cd276af,offer received,2906b810c7d4411798c6938adc9daaa5,F,72,2017-11-17,61000.0,1141,discount,0,1,0,0,1,0,0
106549,95e94202440a43939f3195fc975e7367,offer completed,2298d6c36e964ae4a3e7e9706d1fb8c2,M,50,2017-02-09,43000.0,1422,discount,1,0,1,0,1,0,1
76270,5815952104fd40bb84730a93a750cc82,offer viewed,3f207df678b143eea3cee63160fa8bed,O,54,2017-05-14,87000.0,1328,informational,1,0,0,0,0,0,0
90146,ac83538a4c4a4173ae663b656fcb14ec,offer received,fafdcd668e3743c1bb461111dcafc2a4,M,28,2018-03-01,36000.0,1037,discount,0,0,1,0,1,0,0
132957,93534b7c68e44474a67b2c41fe95ecaa,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,M,87,2015-04-10,55000.0,2093,bogo,1,0,1,1,0,0,0
118024,4a74c87b845e49b7b73a2674e09798e5,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,M,82,2018-07-16,47000.0,900,discount,1,0,1,0,1,0,0
12231,b8a0e096e34e4281baebc5aead2de945,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,73,2018-07-02,61000.0,914,bogo,1,1,0,1,0,0,0
39149,f3934f05d51f47c7a470661cbb774075,offer viewed,ae264e3637204a6fb9bb56bc8210ddfd,M,55,2016-05-08,40000.0,1699,bogo,1,0,1,1,0,0,0
85348,de90515e9f174ddfb8b4c677448286ef,offer completed,fafdcd668e3743c1bb461111dcafc2a4,M,55,2017-09-11,74000.0,1208,discount,1,0,1,0,1,0,1


### drop informational offers

In [24]:
# drop all the informational rows
df.drop(df.index[df['offer_type'] == "informational"], inplace=True)
df.head()

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,0,0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,0,0
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,1,0
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0,0,0
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0,0,0


In [25]:
df.offer_type.value_counts()

bogo        63834
discount    62311
Name: offer_type, dtype: int64

In [26]:
df.shape

(126145, 16)

## completed count

In [27]:
#filter for completed ones and non-completed ones
comp=df[df['event']=='offer completed']
not_comp=df[df['event']!='offer completed']

In [28]:
comp

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,1,0
5,e2127556f4f64592b11af22de27a7932,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0,1,0
10,389bc3fa690240e798340f5a15918d5c,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,65,2018-02-09,53000.0,1057,bogo,1,0,1,1,0,1,0
19,868317b9be554cb18e50bc68484749a2,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,96,2017-11-17,89000.0,1141,bogo,1,1,0,1,0,1,0
21,f082d80f0aac47a99173ba8ef8fc1909,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,40,2016-07-09,33000.0,1637,bogo,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148767,3027ea16075548fb8e761021b3a4f684,offer completed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,56,2016-07-28,75000.0,1618,bogo,1,0,1,1,0,1,0
148775,3a298dbf99ef4a23b9b46a1500e56101,offer completed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,58,2016-09-12,118000.0,1572,bogo,1,0,1,1,0,1,0
148780,9a771d17217c4cce87c1abeca24b70bb,offer completed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,68,2016-09-03,39000.0,1581,bogo,1,0,1,1,0,1,0
148784,990080173c3f4871bb8634a9b50e118a,offer completed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,76,2017-11-18,75000.0,1140,bogo,1,0,1,1,0,1,0


In [29]:
#count how many offers completed for each user
comp_count=comp.groupby('person id',as_index=False)['event'].count()
comp_count

,person id,event
0,0009655768c64bdeb2e877511632db8f,3
1,0011e0d4e6b944f998e987f904e8c1e5,3
2,0020c2b971eb4e9188eac86d93036a77,3
3,0020ccbbb6d84e358d3414a3ff76cffd,3
4,003d66b6608740288d6cc97a6903f4f0,3
...,...,...
11981,fff29fb549084123bd046dbc5ceb4faa,6
11982,fff3ba4757bd42088c044ca26d73817a,3
11983,fff7576017104bcc8677a8d63322b5e1,3
11984,fffad4f4828548d1b5583907f2e9906b,3


In [30]:
comp_count[comp_count['event']==0]

,person id,event


In [31]:
#make sure it's done right. 11986 unique customers who redeemed offers
comp_count['person id'].value_counts()

4a3579f62ba94e8eb1cc6657d51a5deb    1
2520c1ca400d43f4bf52ed6f482a6a75    1
98d7c07f21064a5bad8c1ac3262939ca    1
970d7af2030742f2a098184bea0da030    1
bd7b13b7520b4bd1aed39c026ebb9116    1
                                   ..
e61a428312024ef493facecdbf5188cb    1
2afddead3edc4fcd9c6c1840c3de2950    1
b23fd66b4bd4463b8382ff9007f7f861    1
31e36de1923e40dfb2f37ccd67989c7a    1
f92d8e8c8055406ea37c07b77ce505db    1
Name: person id, Length: 11986, dtype: int64

In [32]:
not_comp

,person id,event,offer id,gender,age,became_member_on,income,member_since,offer_type,redeemed,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,0,0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,2017-05-09,100000.0,1333,bogo,1,1,0,1,0,0,0
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0,0,0
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,2018-04-26,70000.0,981,bogo,1,0,1,1,0,0,0
6,389bc3fa690240e798340f5a15918d5c,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,65,2018-02-09,53000.0,1057,bogo,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,2017-10-13,94000.0,1176,bogo,0,0,1,1,0,0,0
148801,3f3494f3234e4206a279662bef81fe85,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0,1,0,0,0
148802,3f3494f3234e4206a279662bef81fe85,offer viewed,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,2014-07-15,60000.0,2362,bogo,0,1,0,1,0,0,0
148803,3045af4e98794a04a5542d3eac939b1f,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,2016-10-20,78000.0,1534,bogo,0,1,0,1,0,0,0


In [33]:
#dummies for event. 0 means not redeemed
not_comp_count=not_comp.groupby('person id',as_index=False)['event'].count()
not_comp_count['event']=[0 for i in not_comp_count['event']]
not_comp_count

,person id,event
0,0009655768c64bdeb2e877511632db8f,0
1,0011e0d4e6b944f998e987f904e8c1e5,0
2,0020c2b971eb4e9188eac86d93036a77,0
3,0020ccbbb6d84e358d3414a3ff76cffd,0
4,003d66b6608740288d6cc97a6903f4f0,0
...,...,...
14759,fff3ba4757bd42088c044ca26d73817a,0
14760,fff7576017104bcc8677a8d63322b5e1,0
14761,fff8957ea8b240a6b5e634b6ee8eafcf,0
14762,fffad4f4828548d1b5583907f2e9906b,0


In [34]:
#check to see done correctly or not 
not_comp_count[not_comp_count['person id']=='78afa995795e4d85b5d9ceeca43f5fef']

,person id,event
6929,78afa995795e4d85b5d9ceeca43f5fef,0


In [35]:
#put the two together 
cdf=pd.concat([comp_count, not_comp_count], axis=0)
cdf

,person id,event
0,0009655768c64bdeb2e877511632db8f,3
1,0011e0d4e6b944f998e987f904e8c1e5,3
2,0020c2b971eb4e9188eac86d93036a77,3
3,0020ccbbb6d84e358d3414a3ff76cffd,3
4,003d66b6608740288d6cc97a6903f4f0,3
...,...,...
14759,fff3ba4757bd42088c044ca26d73817a,0
14760,fff7576017104bcc8677a8d63322b5e1,0
14761,fff8957ea8b240a6b5e634b6ee8eafcf,0
14762,fffad4f4828548d1b5583907f2e9906b,0


In [36]:
cdf[cdf['person id']=='0009655768c64bdeb2e877511632db8f']

,person id,event
0,0009655768c64bdeb2e877511632db8f,3
0,0009655768c64bdeb2e877511632db8f,0


In [37]:
#group by person id to remove the rows for the events (received, viewed) for the completed events
edf=cdf.groupby('person id',as_index=False).max()
edf = edf.rename(columns={'event': 'completed'})
edf

,person id,completed
0,0009655768c64bdeb2e877511632db8f,3
1,0011e0d4e6b944f998e987f904e8c1e5,3
2,0020c2b971eb4e9188eac86d93036a77,3
3,0020ccbbb6d84e358d3414a3ff76cffd,3
4,003d66b6608740288d6cc97a6903f4f0,3
...,...,...
14759,fff3ba4757bd42088c044ca26d73817a,3
14760,fff7576017104bcc8677a8d63322b5e1,3
14761,fff8957ea8b240a6b5e634b6ee8eafcf,0
14762,fffad4f4828548d1b5583907f2e9906b,3


In [38]:
edf[edf['person id']=='0009655768c64bdeb2e877511632db8f']

,person id,completed
0,0009655768c64bdeb2e877511632db8f,3


# Final dataframe

In [39]:
df.columns

Index(['person id', 'event', 'offer id', 'gender', 'age', 'became_member_on',
       'income', 'member_since', 'offer_type', 'redeemed', 'female', 'male',
       'rc_bogo', 'rc_discount', 'rd_bogo', 'rd_discount'],
      dtype='object')

## group by person id

In [40]:
gdf=df[['person id' ,'redeemed', 'age', 'income', 'member_since', 'female', 'male', 'rc_bogo','rc_discount', 'rd_bogo','rd_discount']].groupby(['person id'],as_index=False).max()
gdf

,person id,redeemed,age,income,member_since,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
0,0009655768c64bdeb2e877511632db8f,1,33,72000.0,1351,0,1,1,1,1,1
1,0011e0d4e6b944f998e987f904e8c1e5,1,40,57000.0,1088,0,0,1,1,1,1
2,0020c2b971eb4e9188eac86d93036a77,1,59,90000.0,1764,1,0,1,1,1,1
3,0020ccbbb6d84e358d3414a3ff76cffd,1,24,60000.0,1512,1,0,1,1,1,1
4,003d66b6608740288d6cc97a6903f4f0,1,26,73000.0,1290,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
14759,fff3ba4757bd42088c044ca26d73817a,1,69,83000.0,1930,1,0,1,1,1,1
14760,fff7576017104bcc8677a8d63322b5e1,1,71,73000.0,1158,0,1,1,1,1,1
14761,fff8957ea8b240a6b5e634b6ee8eafcf,0,71,56000.0,1048,0,1,1,1,0,0
14762,fffad4f4828548d1b5583907f2e9906b,1,34,34000.0,1439,0,1,1,0,1,0


In [41]:
gdf[gdf['person id']=='0009655768c64bdeb2e877511632db8f']

,person id,redeemed,age,income,member_since,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
0,0009655768c64bdeb2e877511632db8f,1,33,72000.0,1351,0,1,1,1,1,1


## put things togther

In [42]:
fdf=pd.merge(edf,gdf,on='person id')
fdf

,person id,completed,redeemed,age,income,member_since,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
0,0009655768c64bdeb2e877511632db8f,3,1,33,72000.0,1351,0,1,1,1,1,1
1,0011e0d4e6b944f998e987f904e8c1e5,3,1,40,57000.0,1088,0,0,1,1,1,1
2,0020c2b971eb4e9188eac86d93036a77,3,1,59,90000.0,1764,1,0,1,1,1,1
3,0020ccbbb6d84e358d3414a3ff76cffd,3,1,24,60000.0,1512,1,0,1,1,1,1
4,003d66b6608740288d6cc97a6903f4f0,3,1,26,73000.0,1290,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14759,fff3ba4757bd42088c044ca26d73817a,3,1,69,83000.0,1930,1,0,1,1,1,1
14760,fff7576017104bcc8677a8d63322b5e1,3,1,71,73000.0,1158,0,1,1,1,1,1
14761,fff8957ea8b240a6b5e634b6ee8eafcf,0,0,71,56000.0,1048,0,1,1,1,0,0
14762,fffad4f4828548d1b5583907f2e9906b,3,1,34,34000.0,1439,0,1,1,0,1,0


In [43]:
# users that didn't redeem any offer
fdf[fdf['redeemed']==0]

,person id,completed,redeemed,age,income,member_since,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
13,00857b24b13f4fe0ad17b605f00357f5,0,0,71,41000.0,1166,0,1,1,1,0,0
14,008d7088107b468893889da0ede0df5c,0,0,24,42000.0,1209,0,1,1,1,0,0
19,00aee28bbb3848dd8a31f0c91dc267dd,0,0,21,59000.0,1091,0,1,1,1,0,0
23,00b901d68f8f4fd68075184cd0f772d2,0,0,68,76000.0,924,1,0,1,1,0,0
29,00c32a104f0c4065b5b552895fb22e34,0,0,33,42000.0,1106,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14748,ffc77312ccda49fca19506f1cdf44923,0,0,37,70000.0,1384,0,1,1,1,0,0
14751,ffd810f301ad42d6bfe98c4ba6ee4125,0,0,65,52000.0,1051,0,1,1,1,0,0
14756,ffed75d3abc64b488982f50ed12878b5,0,0,20,44000.0,1023,0,1,1,0,0,0
14757,fff0f0aac6c547b9b263080f09a5586a,0,0,67,80000.0,1296,0,1,1,0,0,0


In [44]:
# users that redeemed atleast one offer
fdf[fdf['redeemed']==1]

,person id,completed,redeemed,age,income,member_since,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
0,0009655768c64bdeb2e877511632db8f,3,1,33,72000.0,1351,0,1,1,1,1,1
1,0011e0d4e6b944f998e987f904e8c1e5,3,1,40,57000.0,1088,0,0,1,1,1,1
2,0020c2b971eb4e9188eac86d93036a77,3,1,59,90000.0,1764,1,0,1,1,1,1
3,0020ccbbb6d84e358d3414a3ff76cffd,3,1,24,60000.0,1512,1,0,1,1,1,1
4,003d66b6608740288d6cc97a6903f4f0,3,1,26,73000.0,1290,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14758,fff29fb549084123bd046dbc5ceb4faa,6,1,59,93000.0,1219,1,0,1,1,1,1
14759,fff3ba4757bd42088c044ca26d73817a,3,1,69,83000.0,1930,1,0,1,1,1,1
14760,fff7576017104bcc8677a8d63322b5e1,3,1,71,73000.0,1158,0,1,1,1,1,1
14762,fffad4f4828548d1b5583907f2e9906b,3,1,34,34000.0,1439,0,1,1,0,1,0


In [45]:
fdf.sample(30)

,person id,completed,redeemed,age,income,member_since,female,male,rc_bogo,rc_discount,rd_bogo,rd_discount
14046,f3a513ed02234f539b89e8e8df1e49c0,4,1,80,50000.0,1265,0,1,1,1,1,1
2247,286424d90dee4e23a07e1f374c69a89e,4,1,63,84000.0,1381,1,0,1,0,1,0
4820,533342f427824eb59adc4aa9949fe666,5,1,42,56000.0,1795,1,0,1,1,1,1
11838,ccfba2843e944de0aadd187c61cd2774,0,0,82,57000.0,1768,0,1,1,1,0,0
1828,20b3771f1f144a8d95610a19cd65d8d3,3,1,70,58000.0,1097,0,1,1,1,1,1
10683,b7e2d5ab4ab24229bf1aa74f2b2eeb4d,2,1,70,83000.0,1106,1,0,0,1,0,1
7143,7be78ebad1ac4467bbe93813fe0b7d28,2,1,67,52000.0,2513,0,1,1,1,0,1
1118,13bbddb500e54ab392c8da22ae7a41cd,4,1,62,78000.0,2048,1,0,1,1,1,1
1890,21fa118bc6b84650976fed5df7d3040c,1,1,68,58000.0,1464,0,1,0,1,0,1
2222,27fc2e4cd78a4a258b9ef58be528beda,0,0,30,31000.0,940,1,0,1,1,0,0
